# What does this model do?

1. Predict utilization counts for each category (e.g., PCP visits, Rx fills, inpatient stays).

2. Multiply predicted counts by unit costs from insurance data to estimate total allowed cost.
This is then run through a benefit calculator to estimate patient out-of-pocket (OOP) costs.

Model is trained on MEPS data: https://meps.ahrq.gov/mepsweb/data_stats/download_data_files.jsp

# Imports

In [34]:
import pandas as pd
import polars as pl # using polars library for large .xlsx files
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, root_mean_squared_error

Skip to "Load MEPS Data" if h243.xlsx has already been converted to .csv and .parquet.

In [ ]:
# Define schema for problematic columns
schema_overrides = {
    'HRWG53X': pl.Float64,  # Use float instead of int for decimal values
    # Add other problematic columns as needed
}

df = pl.read_excel(
    "/Users/orenj/Desktop/Projects/open-coverage/python-models/data/h243.xlsx",
    schema_overrides=schema_overrides
)

df.head()

Convert to faster formats for later use

In [10]:
df.write_csv("/Users/orenj/Desktop/Projects/open-coverage/python-models/data/h243.csv")  # Much faster for future loads
df.write_parquet("/Users/orenj/Desktop/Projects/open-coverage/python-models/data/h243.parquet")  # Even faster, compressed

# Load MEPS Data

ChatGPT: Use consolidated from year t and counts from year t+1 if you're predicting future utilization (prospective), which is better for real-world use.

We are using consolidated from 2022 and counts from 2023.

## Consolidated Data

Load MEPS Consolidated File (person-level features + target)

In [3]:
df_cons = pd.read_parquet("/Users/orenj/Desktop/Projects/open-coverage/python-models/data/h243/h243.parquet")

df_cons.head()

,DUID,PID,DUPERSID,PANEL,DATAYEAR,FAMID31,FAMID42,FAMID53,FAMID22,FAMIDYR,...,RXOSR22,RXPTR22,RXOTH22,PERWT22F,FAMWT22F,FAMWT22C,SAQWT22F,DIABW22F,VARSTR,VARPSU
0,2460002,101,2460002101,24,2022,A,A,A,A,A,...,0,0,0,5728.309495,5232.211986,5232.211986,3994.687140,6034.636755,2082,1
1,2460006,101,2460006101,24,2022,A,A,A,A,A,...,0,0,0,15648.881461,16017.881691,16017.881691,0.000000,0.000000,2001,4
2,2460006,102,2460006102,24,2022,A,A,A,A,A,...,0,0,0,14123.720178,16017.881691,12580.731740,0.000000,0.000000,2001,4
3,2460010,101,2460010101,24,2022,A,A,A,A,A,...,0,5288,299,16982.054917,21905.758877,21905.758877,0.000000,0.000000,2038,3
4,2460018,101,2460018101,24,2022,A,A,A,A,A,...,0,10,0,10682.619947,11344.291012,11344.291012,17152.439412,0.000000,2041,1


## Aggregate Event Files

Load data files for **Counts per Person**

In [9]:
event_files = {
    "pcp_visits": "/Users/orenj/Desktop/Projects/open-coverage/python-models/data/h248/h248g.xlsx",            # Office-based visits
    "outpatient_visits": "/Users/orenj/Desktop/Projects/open-coverage/python-models/data/h248/h248f.xlsx",     # Outpatient visits
    "er_visits": "/Users/orenj/Desktop/Projects/open-coverage/python-models/data/h248/h248e.xlsx",             # Emergency Room visits
    "inpatient_admits": "/Users/orenj/Desktop/Projects/open-coverage/python-models/data/h248/h248d.xlsx",      # Inpatient stays
    "home_health_visits": "/Users/orenj/Desktop/Projects/open-coverage/python-models/data/h248/h248h.xlsx",    # Home health visits
    "rx_fills": "/Users/orenj/Desktop/Projects/open-coverage/python-models/data/h248/h248a.xlsx",              # Prescription fills
    "dental_visits": "/Users/orenj/Desktop/Projects/open-coverage/python-models/data/h248/h248b.xlsx",         # Dental visits
    "equipment_purchases": "/Users/orenj/Desktop/Projects/open-coverage/python-models/data/h248/h248c.xlsx"    # Medical equipment/supplies
}

# Count records per DUPERSID for each event type
counts_dict = {}
for feat_name, filepath in event_files.items():
    # Use pandas with usecols parameter (which works correctly)
    df_evt = pd.read_excel(filepath, usecols=["DUPERSID"])
    
    # Count occurrences per DUPERSID
    counts = df_evt.groupby("DUPERSID").size().rename(feat_name)
    counts_dict[feat_name] = counts

# Merge all counts into a single DataFrame
counts_df = (
    pd.concat(counts_dict.values(), axis=1)
      .fillna(0)
      .reset_index()
)

## Merge consolidated with counts

Doing an inner join on DUPERSID.

DUPERSID: AHRQ‑generated person identifier (derived from dwelling/unit and person IDs). It uniquely identifies a respondent within a year, and from 2018 onward it’s constructed to be unique across panels/year.

In [17]:
df = (
    df_cons
      .merge(counts_df, on="DUPERSID", how="inner")
      .fillna(0)
)
df.head()

,DUID,PID,DUPERSID,PANEL,DATAYEAR,FAMID31,FAMID42,FAMID53,FAMID22,FAMIDYR,...,VARSTR,VARPSU,pcp_visits,outpatient_visits,er_visits,inpatient_admits,home_health_visits,rx_fills,dental_visits,equipment_purchases
0,2790002,101,2790002101,27,2022,A,A,A,A,A,...,2019,1,3.0,1.0,0.0,0.0,0.0,9.0,0.0,0.0
1,2790002,102,2790002102,27,2022,A,A,A,A,A,...,2019,1,1.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0
2,2790004,101,2790004101,27,2022,A,A,A,A,A,...,2084,1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
3,2790006,101,2790006101,27,2022,A,A,A,A,A,...,2113,1,3.0,0.0,0.0,0.0,1.0,26.0,0.0,1.0
4,2790011,101,2790011101,27,2022,A,A,A,A,A,...,2029,3,22.0,0.0,0.0,0.0,0.0,5.0,3.0,0.0


In [18]:
df.shape

(7022, 1428)

# Select Features and Target Variables

| Feature                     | MEPS Variable | Description                                          | Source                                  |
| --------------------------- | ------------- | ---------------------------------------------------- | --------------------------------------- |
| **Person ID**               | `DUPERSID`    | Unique person identifier (for merges)                | ([Medical Expenditure Panel Survey][1]) |
| **Age**                     | `AGE22X`      | Age as of 12/31/2022 (imputed)                       | ([Medical Expenditure Panel Survey][1]) |
| **Sex**                     | `ADSEX42`     | Survey‑reported gender (1 = Male, 2 = Female)        | ([Medical Expenditure Panel Survey][1]) |
| **Insurance Status**        | `INSCOV22`    | 1 = Any private • 2 = Public only • 3 = Uninsured    | ([Medical Expenditure Panel Survey][1]) |
| **BMI**                     | `ADBMI42`     | Body mass index (>17) from Round 4/5                 | ([Medical Expenditure Panel Survey][1]) |
| **Smoking Frequency**       | `ADOSTP42`    | “How often use tobacco” in past 12 months            | ([Medical Expenditure Panel Survey][1]) |
| **Alcohol Consumption**     | `ADASKALC42`  | “12 months: alcohol consumption”                     | ([Medical Expenditure Panel Survey][1]) |
| **Exercise**                | `ADDAYEXER42` | Average days/week of moderate/vigorous exercise      | ([Medical Expenditure Panel Survey][1]) |
| **Chronic Condition Count** | `TOTCHRON`    | Number of chronic conditions                         | ([Medical Expenditure Panel Survey][1]) |
| **Total Expenditures**      | `TOTEXP22`    | Sum of all payments in 2022 (your regression target) | ([Medical Expenditure Panel Survey][1]) |

[1]: https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_codebook.jsp?PUFId=H243 "Medical Expenditure Panel Survey PUF Codebook"


In [24]:
feature_cols = [
    "AGE22X", "ADSEX42", "INSCOV22", "ADBMI42", 
    "ADOSTP42", "ADASKALC42", "ADDAYEXER42",
    #"TOTCHRON",  can't find this feature right now
    "pcp_visits", 
    #"prior_year_specialist_visits", can't find this feature right now
    "er_visits", 
    "inpatient_admits",
]
count_targets = [
    "pcp_visits", "outpatient_visits", "er_visits",
    "inpatient_admits", "home_health_visits",
    "rx_fills", "dental_visits", "equipment_purchases"
]

X = df[feature_cols]
y = df[count_targets].clip(lower=0)

# Train/Test Split

In [25]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Preprocessor for Categorical Features

In [28]:
preprocessor = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), ["AGE22X", "INSCOV22"])
], remainder="passthrough")

# Fit one Poisson GBM per count target

In [29]:
models = {}
preds = pd.DataFrame(index=X_test.index)

for target in count_targets:
    pipe = Pipeline([
        ("preproc", preprocessor),
        ("model", HistGradientBoostingRegressor(
            loss="poisson",
            max_iter=200,
            learning_rate=0.1,
            random_state=42
        ))
    ])
    pipe.fit(X_train, y_train[target])
    models[target] = pipe
    preds[target] = pipe.predict(X_test).clip(min=0)

# Evaluate Each Count Model

In [35]:
metrics = []
for target in count_targets:
    mae = mean_absolute_error(y_test[target], preds[target])
    rmse = root_mean_squared_error(y_test[target], preds[target])
    metrics.append({
        "feature": target,
        "MAE": mae,
        "RMSE": rmse
    })

metrics_df = pd.DataFrame(metrics).set_index("feature")
print("Count Models Evaluation (per-category):")
print(metrics_df)

Count Models Evaluation (per-category):
                           MAE       RMSE
feature                                  
pcp_visits            0.153970   1.781011
outpatient_visits     1.617641   5.118304
er_visits             0.014882   0.151486
inpatient_admits      0.002534   0.032686
home_health_visits    0.518352   2.032774
rx_fills             10.019091  18.140352
dental_visits         1.146292   1.711277
equipment_purchases   0.671357   0.905474


# Compose Total Allowed Cost (placeholder unit costs)

Replace `unit_costs` with your actual costs per category

In [36]:
unit_costs = {
    "pcp_visits":        120,
    "outpatient_visits": 250,
    "er_visits":         1600,
    "inpatient_admits":  18000,
    "home_health_visits": 100,
    "rx_fills":           85,
    "dental_visits":      200,
    "equipment_purchases": 500
}

Compute composed allowed cost for each test record

In [37]:
composed = preds.copy()
for cat, cost in unit_costs.items():
    composed[cat] *= cost

composed["total_allowed_pred"] = composed.sum(axis=1)

Evaluate composed total allowed cost against MEPS target (TOTEXP as proxy)

Note: TOTEXP includes all payments – check alignment with your unit_costs definitions

In [41]:
true_total = df.loc[X_test.index, "TOTEXP22"]
mae_tot = mean_absolute_error(true_total, composed["total_allowed_pred"])
rmse_tot = root_mean_squared_error(true_total, composed["total_allowed_pred"])
print(f"\nComposed Total Allowed Cost MAE: ${mae_tot:,.2f}")
print(f"Composed Total Allowed Cost RMSE: ${rmse_tot:,.2f}")


Composed Total Allowed Cost MAE: $7,844.22
Composed Total Allowed Cost RMSE: $25,545.28
